In [3]:
require(data.table)
require(tidyverse)

## 1. Read data 

In [4]:
dat.mat <- fread("gzcat ../dat/1910_v2//long_matrix_w_transcripts.txt.gz") %>% filter(!is.na(tr.idx))
dim(dat.mat)
head(dat.mat)
# dat.mat$cell <- NULL #no need cell id

[1] 18758757        7

seq,start,end,cell,value,tr.idx,gene
<chr>,<int>,<int>,<chr>,<int>,<int>,<chr>
chr10,100027284,100028604,Islet1fresh_AGACACCTAAGAGGCAGCGTAAGAAGGCG,2,9209,LOXL4
chr10,100027284,100028604,Islet1fresh_AGACACCTACTCGCTAGTAAGGAGCAGGA,2,9209,LOXL4
chr10,100027284,100028604,Islet1fresh_AGACACCTACTGAGCGATCTGAGTGTACT,2,9209,LOXL4
chr10,100027284,100028604,Islet1fresh_AGACACCTATCTCAGGTATCCTCTCCTAT,2,9209,LOXL4
chr10,100027284,100028604,Islet1fresh_AGACACCTATGCGCAGAAGGAGTAGTACT,1,9209,LOXL4
chr10,100027284,100028604,Islet1fresh_AGACACCTCGAGGCTGCTCTCTATGGTTG,2,9209,LOXL4


 Concepts: 
1. `promoter region`: -500bp + 500bp for all TSS in gencode 
2. `promoter Peaks`: peaks that overlap promoter region

In [7]:
dat.mat <- dat.mat %>% mutate(seq = sub("chr", "", seq)) %>% mutate(prom_peak = paste0(seq, 
    ":", start, "-", end)) %>% select(prom_peak,cell)%>%distinct()
dim(dat.mat)
dat.mat %>%head(1)

[1] 18758757        2

prom_peak,cell
<chr>,<chr>
10:100027284-100028604,Islet1fresh_AGACACCTAAGAGGCAGCGTAAGAAGGCG


In [8]:
dat.mat <- dat.mat %>% left_join(fread("../dat/1910_v2/islet.cluster_labels.filt.txt") %>% 
    select(index, cluster_name), by = c(cell = "index"))
dat.mat %>% head(1)
dat.mat %>% dim

prom_peak,cell,cluster_name
<chr>,<chr>,<chr>
10:100027284-100028604,Islet1fresh_AGACACCTAAGAGGCAGCGTAAGAAGGCG,alpha_1


[1] 18758757        3

In [5]:
# get all cells 
dat.all.cells <- table((fread('../dat/1910_v2/islet.cluster_labels.filt.txt'))$cluster) # table for all cells
dat.all.cells
sum(dat.all.cells)
length(unique(dat.mat$cell))


     acinar     alpha_1     alpha_2      beta_1      beta_2     delta_1 
         46        5191        1027        4204        3394         478 
    delta_2      ductal endothelial       gamma      immune    stellate 
        232          80         118         260         140         128 

[1] 15298

[1] 15298

## 2 Prepare data

In [11]:
# prepare data
setDT(dat.mat)
dat.mat <- dat.mat[, .N, by = .(prom_peak, cluster_name)] %>% arrange(prom_peak, 
    cluster_name)
head(dat.mat, 1)
dat.mat %>% nrow

prom_peak,cluster_name,N
<chr>,<chr>,<int>
1:10002866-10003720,acinar,5


[1] 195396

In [13]:
dat.mat <- dat.mat%>%rename(cluster=cluster_name)

## 3. Fisher's exact test  between subtypes

As long as there is any  peaks in that cell, the peak is open in this cell. 

1. get total alpha 1 and alpha 2 cells 
2. test hit in alpha1 vs hit in alpha2  (create contentigen table)
3. perform [Fisher's exact test](https://en.wikipedia.org/wiki/Fisher%27s_exact_test) or [chi-squared test](https://en.wikipedia.org/wiki/Chi-squared_test)

Input: `dat.mat` and `dat.all.cells`
output: 

peak|n_celltype1|n_celltype2|total_cells|celltype | pval | frac_1 | frac_2| odds


### 3.1 perform fisher's exact test for all prom peaks

In [14]:
tr = "1:10002866-10003720"
celltypes <- c("alpha_1", "alpha_2")

test.dat <- dat.mat %>% filter(prom_peak == tr & cluster %in% celltypes)
test.dat
table.res<- test.dat$N; names(table.res) <- test.dat$cluster
# handle if 0 for one subtype
a = setdiff(celltypes, names(table.res))
table.res[a] <- 0
test.tab <- matrix(c(table.res[celltypes[1]], table.res[celltypes[2]], dat.all.cells[celltypes[1]] - 
    table.res[celltypes[1]], dat.all.cells[celltypes[2]] - table.res[celltypes[2]]), 
    byrow = T, nrow = 2, dimnames = list(expressed = c("Yes", "No"), subtype = celltypes))
test.tab
test.tab[1]
test.tab[2]


f1 <- test.tab[1]/(test.tab[1] + test.tab[2])
f2 <- test.tab[3]/(test.tab[3] + test.tab[4])
l <- ifelse(f1 > f2, "greater", "less")
test.res <- fisher.test(test.tab, alternative = l)
res <- list(pval = test.res$p.value, odds = test.res$estimate, type1_frac = f1, type2_frac = f2)
res

fisher.test(test.tab)

prom_peak,cluster,N
<chr>,<chr>,<int>
1:10002866-10003720,alpha_1,279
1:10002866-10003720,alpha_2,67


,alpha_1,alpha_2
Yes,279,67
No,4912,960


[1] 279

[1] 4912

$pval
[1] 0.08365895

$odds
odds ratio 
 0.8138749 

$type1_frac
[1] 0.05374687

$type2_frac
[1] 0.06523856


	Fisher's Exact Test for Count Data

data:  test.tab
p-value = 0.1565
alternative hypothesis: true odds ratio is not equal to 1
95 percent confidence interval:
 0.6153122 1.0889392
sample estimates:
odds ratio 
 0.8138749 


In [17]:

celltypes <- c('alpha_1','alpha_2')
dat.mat.sub <- dat.mat%>%filter(cluster %in% celltypes)
fun.ftestPerTr <- function(  tr = "1:10002866-10003720",
                             celltypes=c('alpha_1','alpha_2'),
                             dat=dat.mat.sub){
    

test.dat <- dat %>% filter(prom_peak == tr)
 table.res<- test.dat$N; names(table.res) <- test.dat$cluster

    # handle if 0 for one subtype 
    a=setdiff(celltypes,    names(table.res))
    table.res[a]<-0
    
    test.tab <- matrix(c(table.res[celltypes[1]], table.res[celltypes[2]], 
                         dat.all.cells[celltypes[1]]-table.res[celltypes[1]], 
                         dat.all.cells[celltypes[2]]-table.res[celltypes[2]]),
                       byrow =  T,       
                       nrow = 2,
                       dimnames = list(expressed = c("Yes", "No"),
                       subtype = celltypes))

    f1<-test.tab[1]/(test.tab[1]+test.tab[2])
    f2 <- test.tab[3]/(test.tab[3]+test.tab[4])
    l <-ifelse(f1>f2,'greater','less')
    test.res <- fisher.test(test.tab,alternative = l)
    res <- list(
                prom_peak =tr,
                pval=test.res$p.value,
                odds=test.res$estimate,
                type1_frac=f1,
                type2_frac=f2
               )
    res
}

##



system.time(fun.ftestPerTr(celltypes=celltypes))
t(fun.ftestPerTr(celltypes=celltypes))



   user  system elapsed 
  0.019   0.000   0.019 

prom_peak,pval,odds,type1_frac,type2_frac
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1:10002866-10003720,0.08365895,0.8138749,0.05374687,0.06523856


In [18]:
# estimate time
celltypes <- list()
celltypes$alpha <- c("alpha_1", "alpha_2")
celltypes$beta <- c("beta_1", "beta_2")
celltypes$delta <- c("delta_1", "delta_2")
sum(sapply( c("alpha", "beta", "delta"),function(x) 
    length(unique((dat.mat%>% filter(cluster %in% celltypes[[x]]))$prom_peak))))*.029/3600

[1] 0.4467772

In [27]:
dat.mat%>%pull(prom_peak)%>%unique%>%length

[1] 18926

In [19]:
celltypes <- list()
celltypes$alpha <- c("alpha_1", "alpha_2")
celltypes$beta <- c("beta_1", "beta_2")
celltypes$delta <- c("delta_1", "delta_2")

In [24]:
## test run

x<- "alpha"
dat.mat.sub <- dat.mat%>%filter(cluster %in% celltypes[[x]])
all.tr <- unique(dat.mat.sub$prom_peak)
length(all.tr)
do.call(rbind,lapply(all.tr[1:10],function(trr) t(fun.ftestPerTr(dat = dat.mat.sub, 
        tr = trr, celltypes = celltypes[[x]]))))

[1] 18808

prom_peak             pval         odds      type1_frac  type2_frac 
 [1,] 1:10002866-10003720   0.08365895   0.8138749 0.05374687  0.06523856 
 [2,] 1:100110838-100111038 0.1967455    Inf       0.00173377  0          
 [3,] 1:100111215-100112205 0.05610592   1.933956  0.0130996   0.006815969
 [4,] 1:100231527-100232346 0.05999696   2.336528  0.009054132 0.003894839
 [5,] 1:100315017-100316186 0.3611784    1.047567  0.1096128   0.1051607  
 [6,] 1:100434863-100436185 0.02706195   1.250529  0.1165479   0.09542356 
 [7,] 1:100503350-100504585 6.875993e-05 0.7101124 0.1512233   0.2005842  
 [8,] 1:10056643-10058257   0.4368498    0.933159  0.01637449  0.01752678 
 [9,] 1:100598102-100599052 0.2393444    1.088234  0.1144288   0.1061344  
[10,] 1:100643436-100644216 0.4212565    0.878411  0.007705644 0.008763389

In [22]:
require(parallel)

res.transcript_level <- list()
# time consumming task
system.time(for (x in c("alpha", "beta", "delta")) {
    
    dat.mat.sub <- dat.mat %>% filter(cluster %in% celltypes[[x]])
    all.tr <- unique(dat.mat.sub$prom_peak)
    
    res.transcript_level[[x]] <- do.call(rbind, mclapply(all.tr, function(trr) t(fun.ftestPerTr(dat = dat.mat.sub, 
        tr = trr, celltypes = celltypes[[x]])), mc.cores = 10))
})

saveRDS(object = res.transcript_level, file = "../dat/1901/res.prom.peaks.fisher.Rds")

Loading required package: parallel


   user  system elapsed 
186.841  13.627  20.560 

### 3.2 adding gene info

In [23]:
res.transcript_level <- rbind(data.frame(res.transcript_level$alpha,celltype="alpha"),
     data.frame(res.transcript_level$beta,celltype="beta"),
     data.frame(res.transcript_level$delta,celltype="delta"))
head(res.transcript_level)

prom_peak,pval,odds,type1_frac,type2_frac,celltype
<list>,<list>,<list>,<list>,<list>,<fct>
1:10002866-10003720,0.08365895,0.8138749,0.05374687,0.06523856,alpha
1:100110838-100111038,0.1967455,Inf,0.00173377,0,alpha
1:100111215-100112205,0.05610592,1.933956,0.0130996,0.006815969,alpha
1:100231527-100232346,0.05999696,2.336528,0.009054132,0.003894839,alpha
1:100315017-100316186,0.3611784,1.047567,0.1096128,0.1051607,alpha
1:100434863-100436185,0.02706195,1.250529,0.1165479,0.09542356,alpha


[1] 0.4313711

In [33]:
peak_tr.dic <- readRDS('../dat/1910_v2/peak_tr.dic.rds')
peak_tr.dic%>%head(1)

prom_peak,gene
<chr>,<chr>
10:100027284-100028604,LOXL4


In [34]:
head(res.transcript_level) %>% mutate(prom_peak = as.character(prom_peak)) %>% left_join(peak_tr.dic)

Joining, by = "prom_peak"


prom_peak,pval,odds,type1_frac,type2_frac,celltype,gene
<chr>,<list>,<list>,<list>,<list>,<fct>,<chr>
1:10002866-10003720,0.08365895,0.8138749,0.05374687,0.06523856,alpha,LZIC
1:10002866-10003720,0.08365895,0.8138749,0.05374687,0.06523856,alpha,NMNAT1
1:100110838-100111038,0.1967455,Inf,0.00173377,0,alpha,PALMD
1:100111215-100112205,0.05610592,1.933956,0.0130996,0.006815969,alpha,PALMD
1:100231527-100232346,0.05999696,2.336528,0.009054132,0.003894839,alpha,FRRS1
1:100315017-100316186,0.3611784,1.047567,0.1096128,0.1051607,alpha,AGL
1:100434863-100436185,0.02706195,1.250529,0.1165479,0.09542356,alpha,SLC35A3


In [35]:
res.transcript_level <- res.transcript_level %>% mutate(prom_peak = as.character(prom_peak)) %>% 
    left_join(peak_tr.dic)


Joining, by = "prom_peak"


### 3.3 padj

In [6]:
# adjust for p value
m <- c("fdr", "bonferroni", "BY")
names(m) <- c("FDR", "padj.Bonferroni", "FDR.BY")
res.transcript_level.2 <- do.call(rbind, lapply(c("alpha", "beta", "delta"), function(ct) {
    res.genes_level.a.df <- res.transcript_level %>% filter(celltype == ct) %>% select(prom_peak, 
        pval, celltype) %>% distinct %>% as.data.frame
    res.genes_level.a.df %>% nrow %>% print
    
    for (i in 1:3) {
        res.genes_level.a.df[[names(m)[i]]] <- p.adjust(as.numeric(res.genes_level.a.df$pval), 
            method = m[i])
    }
    res.genes_level.a.df
}))
head(res.transcript_level.2, 1)
head(res.transcript_level.2 %>% filter(celltype == "beta"), 1)
head(res.transcript_level.2 %>% filter(celltype == "delta"), 1)

[1] 18808
[1] 18818
[1] 17836


prom_peak,pval,celltype,FDR,padj.Bonferroni,FDR.BY
<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
1:10002866-10003720,0.08365895,alpha,0.1542908,1,1


prom_peak,pval,celltype,FDR,padj.Bonferroni,FDR.BY
<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
1:10002866-10003720,0.410729,beta,0.4897207,1,1


prom_peak,pval,celltype,FDR,padj.Bonferroni,FDR.BY
<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
1:10002866-10003720,0.0953964,delta,0.2856767,1,1


In [11]:
res.transcript_level %>% nrow
res.transcript_level.2 %>% nrow
res.transcript_level %>% left_join(res.transcript_level.2) %>% head(1)
res.transcript_level %>% left_join(res.transcript_level.2) %>% nrow
res.transcript_level <- res.transcript_level %>% left_join(res.transcript_level.2) %>% 
    select(prom_peak, gene, celltype, type1_frac, type2_frac, odds, pval, FDR, padj.Bonferroni, 
        FDR.BY)
fwrite(res.transcript_level, file = "../figures/Fig2/subfigs/fig_2.prom.sub_vs_sub.fisher.csv")

[1] 60782

[1] 55462

Joining, by = c("prom_peak", "pval", "celltype")


prom_peak,pval,odds,type1_frac,type2_frac,celltype,gene,FDR,padj.Bonferroni,FDR.BY
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1:10002866-10003720,0.08365895,0.8138749,0.05374687,0.06523856,alpha,LZIC,0.1542908,1,1


Joining, by = c("prom_peak", "pval", "celltype")


[1] 60782

Joining, by = c("prom_peak", "pval", "celltype")


### 3.4 padj again after filtering peaks only present in peak calling for the tested cell types

In [5]:
res.transcript_level<-fread("../figures/Fig2/subfigs/fig_2.prom.sub_vs_sub.fisher.csv")
res.transcript_level%>%head(1)
res.transcript_level%>%nrow

prom_peak,gene,celltype,type1_frac,type2_frac,odds,pval,FDR,padj.Bonferroni,FDR.BY
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1:10002866-10003720,LZIC,alpha,0.05374687,0.06523856,0.8138749,0.08365895,0.1429331,1,1


[1] 53645

In [12]:
res.transcript_level<-fread("../figures/Fig2/subfigs/fig_2.prom.sub_vs_sub.fisher.csv")
res.transcript_level%>%head(1)
res.transcript_level%>%nrow
res.transcript_level%>%filter(celltype=='delta')%>%pull(prom_peak)%>%unique%>%length

prom_peak,gene,celltype,type1_frac,type2_frac,odds,pval,FDR,padj.Bonferroni,FDR.BY
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1:10002866-10003720,LZIC,alpha,0.05374687,0.06523856,0.8138749,0.08365895,0.1429331,1,1


[1] 53645

[1] 14552

In [10]:
## double check if Only chose the peaks that found in cell types for delta
res.peak%>%head(1)
res.peak%>%filter(grepl('delta',V4))%>%head(1)
res.peak%>%filter(grepl('delta',V4))%>%tail(1)
res.peak%>%filter(grepl('delta',V4))%>%distinct%>%nrow
res.peak%>%filter(grepl('delta',V4))%>%pull(prom_peak)%>%unique%>%length

prom_peak,V4
<chr>,<chr>
1:858933-860557,"alpha_1,alpha_2,beta_1,beta_2,endothelial,stellate"


prom_peak,V4
<chr>,<chr>
1:894409-895129,"alpha_1,alpha_2,beta_1,beta_2,delta_1,delta_2,endothelial,gamma,stellate"


,prom_peak,V4
,<chr>,<chr>
14552,Y:22736918-22738079,"acinar,alpha_1,alpha_2,beta_1,beta_2,delta_1,delta_2,endothelial,gamma,immune,stellate"


[1] 14552

[1] 14552

In [6]:
res.peak <- fread("../dat/1910_v2/islet.merged_peaks.anno.bed")
res.peak %>% head(1)
res.peak %>% nrow
res.peak  %>% mutate(V1 = sub("chr", "", V1)) %>% mutate(prom_peak = paste0(V1, 
    ":", V2, "-", V3))%>%filter(prom_peak %in% res.transcript_level$prom_peak)%>%dim
res.peak<- res.peak  %>% mutate(V1 = sub("chr", "", V1)) %>% mutate(prom_peak = paste0(V1, 
    ":", V2, "-", V3))%>%filter(prom_peak %in% res.transcript_level$prom_peak)%>%select(prom_peak,V4)
res.peak%>%head(1)


V1,V2,V3,V4
<chr>,<int>,<int>,<chr>
chr1,752578,752778,endothelial


[1] 228873

[1] 18085     5

prom_peak,V4
<chr>,<chr>
1:858933-860557,"alpha_1,alpha_2,beta_1,beta_2,endothelial,stellate"


In [30]:
# adjust for p value
m <- c("fdr", "bonferroni", "BY")
names(m) <- c("FDR", "padj.Bonferroni", "FDR.BY")
res.transcript_level.2 <- do.call(rbind, lapply(c("alpha", "beta", "delta"), function(ct) {
    # ct <- 'alpha'
    res.genes_level.a.df <- res.transcript_level %>% filter(celltype == ct) %>% select(prom_peak, 
        pval, celltype) %>% distinct %>% as.data.frame
    res.genes_level.a.df %>% nrow %>% print
    res.genes_level.a.df <- res.genes_level.a.df %>% filter(prom_peak %in% (res.peak %>% 
        filter(grepl(ct, V4)) %>% pull(prom_peak)))
    res.genes_level.a.df %>% nrow %>% print
    
    for (i in 1:3) {
        res.genes_level.a.df[[names(m)[i]]] <- p.adjust(as.numeric(res.genes_level.a.df$pval), 
            method = m[i])
    }
    res.genes_level.a.df
}))
head(res.transcript_level.2, 1)
head(res.transcript_level.2 %>% filter(celltype == "beta"), 1)
head(res.transcript_level.2 %>% filter(celltype == "delta"), 1)

[1] 18808
[1] 17022
[1] 18818
[1] 16927
[1] 17836
[1] 14552


prom_peak,pval,celltype,FDR,padj.Bonferroni,FDR.BY
<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
1:10002866-10003720,0.08365895,alpha,0.1429331,1,1


prom_peak,pval,celltype,FDR,padj.Bonferroni,FDR.BY
<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
1:10002866-10003720,0.410729,beta,0.4732717,1,1


prom_peak,pval,celltype,FDR,padj.Bonferroni,FDR.BY
<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
1:10002866-10003720,0.0953964,delta,0.2429061,1,1


In [45]:
res.transcript_level %>% nrow
res.transcript_level.2 %>% nrow
res.transcript_level %>% head(1)
res.transcript_level.2 %>% left_join(res.transcript_level %>% select(prom_peak, celltype, 
    type1_frac, type2_frac, odds) %>% distinct) %>% nrow
res.transcript_level.2 %>% left_join(res.transcript_level %>% select(prom_peak, celltype, 
    type1_frac, type2_frac, odds) %>% distinct) %>% head(1)

res.transcript_level.2 %>% left_join(res.transcript_level %>% select(prom_peak, celltype, 
    type1_frac, type2_frac, gene, odds) %>% distinct) %>% nrow

res.transcript_level.2 %>% left_join(res.transcript_level %>% select(prom_peak, celltype, 
    type1_frac, type2_frac, gene, odds) %>% distinct) %>% head(1)


res.transcript_level.2 %>% left_join(res.transcript_level %>% select(prom_peak, celltype, 
    type1_frac, type2_frac, gene, odds) %>% distinct) %>% select(prom_peak, gene, 
    celltype, type1_frac, type2_frac, odds, pval, FDR, padj.Bonferroni, FDR.BY) %>% 
    head(1)

res.transcript_level <- res.transcript_level.2 %>% left_join(res.transcript_level %>% 
    select(prom_peak, celltype, type1_frac, type2_frac, gene, odds) %>% distinct) %>% 
    select(prom_peak, gene, celltype, type1_frac, type2_frac, odds, pval, FDR, padj.Bonferroni, 
        FDR.BY)

fwrite(res.transcript_level, file = "../figures/Fig2/subfigs/fig_2.prom.sub_vs_sub.fisher.csv")

[1] 60782

[1] 48501

prom_peak,gene,celltype,type1_frac,type2_frac,odds,pval,FDR,padj.Bonferroni,FDR.BY
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1:10002866-10003720,LZIC,alpha,0.05374687,0.06523856,0.8138749,0.08365895,0.1542908,1,1


Joining, by = c("prom_peak", "celltype")


[1] 48501

...

Joining, by = c("prom_peak", "celltype")


prom_peak,pval,celltype,FDR,padj.Bonferroni,FDR.BY,type1_frac,type2_frac,gene,odds
<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
1:10002866-10003720,0.08365895,alpha,0.1429331,1,1,0.05374687,0.06523856,LZIC,0.8138749


Joining, by = c("prom_peak", "celltype")


prom_peak,gene,celltype,type1_frac,type2_frac,odds,pval,FDR,padj.Bonferroni,FDR.BY
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1:10002866-10003720,LZIC,alpha,0.05374687,0.06523856,0.8138749,0.08365895,0.1429331,1,1


Joining, by = c("prom_peak", "celltype")
